# StableLM-Alpha

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Stability-AI/StableLM//blob/main/notebooks/stablelm-alpha.ipynb)

<img src="https://raw.githubusercontent.com/Stability-AI/StableLM/main/assets/mascot.png?token=GHSAT0AAAAAABWTZAV7EFSADKXWO3HDNKPYZBZ6Z7A"/>

This notebook is designed to let you quickly generate text with the latest StableLM models (**StableLM-Alpha**) using Hugging Face's `transformers` library.

In [ ]:
!nvidia-smi

Mon Jan  8 15:16:23 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install -U pip
!pip install accelerate bitsandbytes torch transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 21.4 MB/s eta 0:00:00


In [ ]:
#@title Setup

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, StoppingCriteria, StoppingCriteriaList

from IPython.display import Markdown, display
def hr(): display(Markdown('---'))
def cprint(msg: str, color: str = "blue", **kwargs) -> None:
    color_codes = {
        "blue": "\033[34m",
        "red": "\033[31m",
        "green": "\033[32m",
        "yellow": "\033[33m",
        "purple": "\033[35m",
        "cyan": "\033[36m",
    }

    if color not in color_codes:
        raise ValueError(f"Invalid info color: `{color}`")

    print(color_codes[color] + msg + "\033[0m", **kwargs)

In [ ]:
#@title Pick Your Model
#@markdown Refer to Hugging Face docs for more information the parameters below: https://huggingface.co/docs/transformers/main/en/main_classes/model#transformers.PreTrainedModel.from_pretrained

# Choose model name
model_name = "stabilityai/stablelm-tuned-alpha-7b" #@param ["stabilityai/stablelm-tuned-alpha-7b", "stabilityai/stablelm-base-alpha-7b", "stabilityai/stablelm-tuned-alpha-3b", "stabilityai/stablelm-base-alpha-3b"]

cprint(f"Using `{model_name}`", color="blue")

# Select "big model inference" parameters
torch_dtype = "float16" #@param ["float16", "bfloat16", "float"]
load_in_8bit = False #@param {type:"boolean"}
device_map = "auto"

cprint(f"Loading with: `{torch_dtype=}, {load_in_8bit=}, {device_map=}`")

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=getattr(torch, torch_dtype),
    load_in_8bit=load_in_8bit,
    device_map=device_map,
    offload_folder="./offload",
)

Using `stabilityai/stablelm-tuned-alpha-7b`
Loading with: `torch_dtype='float16', load_in_8bit=False, device_map='auto'`


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/264 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

...

pytorch_model-00002-of-00004.bin:   0%|          | 0.00/9.77G [00:00<?, ?B/s]

pytorch_model-00003-of-00004.bin:   0%|          | 0.00/9.75G [00:00<?, ?B/s]

pytorch_model-00004-of-00004.bin:   0%|          | 0.00/2.45G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
#@title Generate Text
#@markdown <b>Note: The model response is colored in green</b>

class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        stop_ids = [50278, 50279, 50277, 1, 0]
        for stop_id in stop_ids:
            if input_ids[0][-1] == stop_id:
                return True
        return False

# Process the user prompt
user_prompt = "How do you train your pets" #@param {type:"string"}
if "tuned" in model_name:
    # Add system prompt for chat tuned models
    system_prompt =  """<|SYSTEM|>
- StableLM is a helpful and harmless open-source AI language model developed by StabilityAI, specialized in all things pizza.
- Eager to discuss the art of pizza-making, StableLM will enthusiastically share insights, recipes, and tips about pizza.
- If conversations drift away from pizza, StableLM will cleverly and swiftly steer them back to this delicious topic.- StableLM is more than just an information source on pizza; it's also creative in suggesting pizza recipes, toppings, and baking techniques.
- As always, StableLM is committed to being helpful and harmless, refusing to participate in anything that could harm a human or sour the joy of pizza-making.
- Whether you're a novice or a seasoned pizza enthusiast, StableLM is here to assist you in crafting the perfect pizza, providing short, efficient, and outcome-oriented responses.
"""
    prompt = f"{system_prompt}<|USER|>{user_prompt}<|ASSISTANT|>"
else:
    prompt = user_prompt

# Sampling args
max_new_tokens = 128 #@param {type:"slider", min:32.0, max:3072.0, step:32}
temperature = 0.7 #@param {type:"slider", min:0.0, max:1.25, step:0.05}
top_k = 0 #@param {type:"slider", min:0.0, max:1.0, step:0.05}
top_p = 0.9 #@param {type:"slider", min:0.0, max:1.0, step:0.05}
do_sample = True #@param {type:"boolean"}

cprint(f"Sampling with: `{max_new_tokens=}, {temperature=}, {top_k=}, {top_p=}, {do_sample=}`")
hr()

# Create `generate` inputs
inputs = tokenizer(prompt, return_tensors="pt")
inputs.to(model.device)

# Generate
tokens = model.generate(
    **inputs,
    max_new_tokens=max_new_tokens,
    temperature=temperature,
    top_k=top_k,
    top_p=top_p,
    do_sample=do_sample,
    pad_token_id=tokenizer.eos_token_id,
    stopping_criteria=StoppingCriteriaList([StopOnTokens()])
)

# Extract out only the completion tokens
completion_tokens = tokens[0][inputs['input_ids'].size(1):]
completion = tokenizer.decode(completion_tokens, skip_special_tokens=True)

# Display
print(user_prompt + " ", end="")
cprint(completion, color="green")

Sampling with: `max_new_tokens=128, temperature=0.7, top_k=0, top_p=0.9, do_sample=True`


---

How do you train your pets Training your pets is a great way to keep them safe and happy. Here are some tips to help you train your pets:

1. Start with positive reinforcement: Pets are social animals, so it's important to start training them early on. Give them treats and praise when they are good behaviors.
2. Use positive reinforcement: When your pets do something good, reward them with treats, praise, or a special toy. This positive reinforcement will encourage them to continue their good behavior.
3. Be consistent: Consistency is key when training your pets. Don't give your pets too much freedom, and always reward


## License (Apache 2.0)

Copyright (c) 2023 by [StabilityAI LTD](https://stability.ai/)

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.